## Homework 9

Homework
In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the previous homework.

Download the model files from here:

https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx
With wget:

PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL="${PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL="${PREFIX}/hair_classifier_v1.onnx"
wget ${DATA_URL}
wget ${MODEL_URL}
Question 1
To be able to use this model, we need to know the name of the input and output nodes.

What's the name of the output:

output
sigmoid
softmax
prediction

### Question One

In [7]:
!pip install onnx

In [8]:
import onnx

# Load the ONNX model
model_path = "hair_classifier_v1.onnx"
model = onnx.load(model_path)

# Check input nodes
print("Inputs:")
for input_tensor in model.graph.input:
    print(input_tensor.name)

# Check output nodes
print("\nOutputs:")
for output_tensor in model.graph.output:
    print(output_tensor.name)


Inputs:
input

Outputs:
output


In [9]:
!pip install pillow

In [10]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img


### Question Two


Let's download and resize this image:

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

64x64
128x128
200x200
256x256

**Answer = 200×200**

### Question 3


Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

-10.73
-1.073
1.073
10.73

In [18]:
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image

# download helper
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    return Image.open(BytesIO(buffer))

# preprocess helper (same as in training)
def prepare_image(img, target_size=(200, 200)):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

# Imagenet normalization
mean = np.array([0.485, 0.456, 0.406])
std  = np.array([0.229, 0.224, 0.225])

# download + resize
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img, (200, 200))

# convert to numpy (HWC → float32)
x = np.array(img).astype("float32") / 255.0

# normalize
x_norm = (x - mean) / std

# first pixel, R channel
first_pixel_R = x_norm[0, 0, 0]
first_pixel_R


-1.0732939435925546

### Question 4

**Now let's apply this model to this image. What's the output of the model?**

In [49]:
import onnxruntime as ort
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image

# ----- helpers -----
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    return Image.open(BytesIO(buffer))

def prepare_image(img, target_size=(200, 200)):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

# ----- load model -----
session = ort.InferenceSession("hair_classifier_v1.onnx")
input_name  = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# ----- download + preprocessing -----
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img, (200, 200))

# convert to float32 array
x = np.array(img).astype("float32")

# correct HW2/HW3 normalization: scale to [-1, 1]
x = x / 255.0
x = (x - 0.5) * 2.0

# HWC → CHW
x = np.transpose(x, (2, 0, 1))

# add batch dimension
x = np.expand_dims(x, 0)

# ----- run inference -----
output = session.run([output_name], {input_name: x})[0]
print("Model output:", output)
        

Model output: [[-0.20132379]]


In [31]:
import onnxruntime as ort
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image

# ----- helpers -----
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    return Image.open(BytesIO(buffer))

def prepare_image(img, target_size=(200, 200)):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

# ----- load model -----
session = ort.InferenceSession("hair_classifier_v1.onnx")
input_name  = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# ----- download + preprocessing -----
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img, (200, 200))

# convert to float32 array
x = np.array(img).astype("float32")

# correct HW2/HW3 normalization: scale to [-1, 1]
x = x / 255.0
x = (x - 0.5) * 2.0

# HWC → CHW
x = np.transpose(x, (2, 0, 1))

# add batch dimension
x = np.expand_dims(x, 0)

# ----- run inference -----
output = session.run([output_name], {input_name: x})[0]
print("Model output:", output)


Model output: [[-0.20132379]]


Prepare the lambda code
Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with ipython or Jupyter Notebook by importing the file and invoking the function from this file.

Docker
For the next two questions, we'll use a Docker image that we already prepared. This is the Dockerfile that we used for creating the image:

FROM public.ecr.aws/lambda/python:3.13

COPY hair_classifier_empty.onnx.data .
COPY hair_classifier_empty.onnx .
Note that it uses Python 3.13.

The docker image is published to agrigorev/model-2025-hairstyle:v1.

A few notes:

The image already contains a model and it's not the same model as the one we used for questions 1-4.

### Question 5

Download the base image agrigorev/model-2025-hairstyle:v1. You can do it with docker pull.

So what's the size of this base image?

## Question 6

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is hair_classifier_empty.onnx and it's in the current workdir in the image (see the Dockerfile above for the reference). The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

**-0.10**
